In [2]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd

In [4]:
# Read config parser .ini file with your connection information

Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
dotenv.load_dotenv(Path)

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 

connection_parms

# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZJNb%2BIwEIb%2FSuQ9x3aypVALqNKi7iL1I0voqtqbiQ14ceysxyGlv35NaKT20B56i5x3xo%2FnmfHlc6WjvXSgrJmgBFMUSVNaocxmgh6XN%2FEIReC5EVxbIyfoIAFdTsfAK12zrPFbs5D%2FGgk%2BCo0MsO7HBDXOMMtBATO8ksB8yYrs7palmLLaWW9Lq9Gbks8rOIB0PhD2JQJUwNt6XzNC2rbF7Xds3YaklFJCL0hIHSPf%2BvxzeNMH%2BYTQs2M%2BJEI8f2W7UuY0gs%2BwVqcQsJ%2FLZR7nD8USRVmPem0NNJV0hXR7VcrHxe0JAAKBMnsVhhoGIXADseTg4xTzFjAY264138nSVnXjQ3ccvshaCqLtRoUBzGcTVO%2BU4Oepza5GL2uh5jp7Wm1%2B3A1dsaGL%2FH4B2%2FTX34utOzRDs3rKShT97g2nR8NzgEbOzdGrD0c0HcQ0jelgmQwYTdjZOR4NB39QNAteleG%2Bq%2BzheZCOd7bcKtGxKVGTonjAR40pOq0G625w068%2BeEzednndtvsgYD7LrVblIbqxruL%2BYz8JTroTJeJ1F2Wy4kpnQjgJEDxpbdtrJ7kPS%2B1dIxGZnm59v9bT%2Fw%3D%3D&RelayState=ver%3A1-hint%3A759822674653786-ETMsDgAAAZTW

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [5]:
connection_parms

{'account': 'invistaprod.us-east-2.aws',
 'user': 'chitrakumarsai.chenurivenkata@invista.com',
 'authenticator': 'externalbrowser',
 'database': 'DS_D_WORKSPACE',
 'schema': 'THIRD_PARTY_PATENT',
 'warehouse': 'DS_D_TRAINING_WH',
 'role': 'DS_D_SCIENTIST_FR'}

In [7]:
# session.sql("CREATE SCHEMA IF NOT EXISTS PURCHASING_ANOMALY;").collect()